In [1]:

# Pip install (recommended)

!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.4/591.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 17.9 MB/s eta 0:00:00


# Creating train-val split

In [2]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
train_path_img = "/content/drive/MyDrive/capstonesmt6/ikan/train/images/"
train_path_label = "/content/drive/MyDrive/capstonesmt6/ikan/train/label/"
val_path_img = "/content/drive/MyDrive/capstonesmt6/ikan/val/images/"
val_path_label = "/content/drive/MyDrive/capstonesmt6/ikan/val/label/"
test_path = "/content/drive/MyDrive/capstonesmt6/ikan/test/"

In [ ]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')
            
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
    
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")
    


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    
    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/capstonesmt6/ikan/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 1012 images---


  0%|          | 0/810 [00:00<?, ?it/s]

------ Training data created with 80% split 810 images -------


  0%|          | 0/202 [00:00<?, ?it/s]

------ Testing data created with a total of 202 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.72 🚀 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 31.2/78.2 GB disk)


## Training

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/capstonesmt6/ikan/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/capstonesmt6/ikan/training_results name=ikan 

100% 21.5M/21.5M [00:00<00:00, 218MB/s]
Ultralytics YOLOv8.0.72 🚀 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/capstonesmt6/ikan/dataset.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/capstonesmt6/ikan/training_results, name=ikan, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, clas

## Inferencing

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/capstonesmt6/ikan/training_results/ikan/weights/best.pt conf=0.55 source=/content/drive/MyDrive/capstonesmt6/ikan/test_images

Ultralytics YOLOv8.0.72 🚀 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/10 /content/drive/MyDrive/capstonesmt6/ikan/test_images/mix (1).jpg: 640x640 1 ikan_segar, 1 ikan_tidak_segar, 16.3ms
image 2/10 /content/drive/MyDrive/capstonesmt6/ikan/test_images/mix (10).jpg: 640x640 1 ikan_segar, 2 ikan_tidak_segars, 16.4ms
image 3/10 /content/drive/MyDrive/capstonesmt6/ikan/test_images/mix (2).jpg: 640x640 1 ikan_segar, 1 ikan_tidak_segar, 16.3ms
image 4/10 /content/drive/MyDrive/capstonesmt6/ikan/test_images/mix (3).jpg: 640x640 1 ikan_segar, 1 ikan_tidak_segar, 16.4ms
image 5/10 /content/drive/MyDrive/capstonesmt6/ikan/test_images/mix (4).jpg: 640x640 1 ikan_segar, 1 ikan_tidak_segar, 16.3ms
image 6/10 /content/drive/MyDrive/capstonesmt6/ikan/test_images/mix (5).jpg: 640x640 2 ikan_segars, 1 ikan_tidak_segar, 16.4ms
image 7/10 /content/drive/MyDrive/capstonesmt6/ikan/test_images/mix (6).j

In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/capstonesmt6/ikan/output

### inferencing on video


In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/capstonesmt6/ikan/training_results/ikan/weights/best.pt conf=0.55 source=/content/drive/MyDrive/capstonesmt6/ikan/vid2

Output streaming akan dipotong hingga 5000 baris terakhir.
video 4/8 (21/431) /content/drive/MyDrive/capstonesmt6/ikan/vid2/4.mp4: 384x640 1 ikan_segar, 8.7ms
video 4/8 (22/431) /content/drive/MyDrive/capstonesmt6/ikan/vid2/4.mp4: 384x640 1 ikan_segar, 7.4ms
video 4/8 (23/431) /content/drive/MyDrive/capstonesmt6/ikan/vid2/4.mp4: 384x640 1 ikan_segar, 7.4ms
video 4/8 (24/431) /content/drive/MyDrive/capstonesmt6/ikan/vid2/4.mp4: 384x640 1 ikan_segar, 7.4ms
video 4/8 (25/431) /content/drive/MyDrive/capstonesmt6/ikan/vid2/4.mp4: 384x640 (no detections), 8.3ms
video 4/8 (26/431) /content/drive/MyDrive/capstonesmt6/ikan/vid2/4.mp4: 384x640 (no detections), 8.8ms
video 4/8 (27/431) /content/drive/MyDrive/capstonesmt6/ikan/vid2/4.mp4: 384x640 (no detections), 9.0ms
video 4/8 (28/431) /content/drive/MyDrive/capstonesmt6/ikan/vid2/4.mp4: 384x640 1 ikan_segar, 8.3ms
video 4/8 (29/431) /content/drive/MyDrive/capstonesmt6/ikan/vid2/4.mp4: 384x640 (no detections), 8.5ms
video 4/8 (30/431) /content/d

In [ ]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/capstonesmt6/ikan/output

In [ ]:
# !yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/football2/weights/best.pt conf=0.5 source=/content/drive/MyDrive/yolov8/videos

In [ ]:
# !cp -r /content/runs/detect/predict3 /content/drive/MyDrive/yolov8/output

## Export the model -- In progress

Export a YOLOv8 model to any supported format with the `format` argument, i.e. `format=onnx`.

Format                  | `format=...**bold text**`         | Model
---                     | ---                       | ---
PyTorch                 | -                         | yolov8n.pt
TorchScript             | `torchscript`             | yolov8n.torchscript
ONNX                    | `onnx`                    | yolov8n.onnx
OpenVINO                | `openvino`                | yolov8n_openvino_model/
TensorRT                | `engine`                  | yolov8n.engine
CoreML                  | `coreml`                  | yolov8n.mlmodel
TensorFlow SavedModel   | `saved_model`             | yolov8n_saved_model/
TensorFlow GraphDef     | `pb`                      | yolov8n.pb
TensorFlow Lite         | `tflite`                  | yolov8n.tflite
TensorFlow Edge TPU     | `edgetpu`                 | yolov8n_edgetpu.tflite
TensorFlow.js           | `tfjs`                    | yolov8n_web_model/
PaddlePaddle            | `paddle`                  | yolov8n_paddle_model/

In [ ]:
!yolo mode=export model=/content/drive/MyDrive/capstonesmt6/ikan/training_results/ikan/weights/best.pt format=tflite

Ultralytics YOLOv8.0.72 🚀 Python-3.9.16 torch-2.0.0+cu118 CPU
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from /content/drive/MyDrive/capstonesmt6/ikan/training_results/ikan/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (21.5 MB)
requirements: YOLOv8 requirements "onnx2tf>=1.7.7" "sng4onnx>=1.0.1" "onnxsim>=0.4.17" "onnx_graphsurgeon>=0.3.26" "tflite_support" "onnxruntime" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.0/350.0 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!yolo mode=export model=/content/drive/MyDrive/capstonesmt6/ikan/training_results/ikan/weights/best.pt format=h5

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/cfg/__init__.py", line 397, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/engine/model.py", line 340, in export
    return Exporter(overrides=args, _callbacks=self.callbacks)(model=self.model)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/engine/exporter.py", line 155, in __call__
    raise ValueError(f"Invalid export format='{format}'. Valid formats are {fmts}")
ValueError: Invalid export format='h5'. Valid formats are ('torchscript', 'onnx', 'openvino', 'engine', 'coreml', 'saved_model', 'pb', 'tflite', 'edgetpu', 'tfjs', 'paddle')
